In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
df = pd.read_stata('Census_2016_Individual_PUMF.dta')

In [3]:
census = df.copy()

## Part One: Filtering the Data

In [4]:
# Filter by age group
df = df[(df['agegrp'] == '20 to 24 years') | (df['agegrp'] == '25 to 29 years') | (df['agegrp'] == '30 to 34 years') 
   | (df['agegrp'] == '35 to 39 years') | (df['agegrp'] == '40 to 44 years')| (df['agegrp'] == '45 to 49 years')
  | (df['agegrp'] == '50 to 54 years') | (df['agegrp'] == '55 to 59 years') | (df['agegrp'] == '60 to 64 years')]

# Then filter by highest degree earned
df = df[(df['hdgree'] != 'no certificate, diploma or degree') & (df['hdgree'] != 'secondary (high) school diploma or equivalency certificate')
 & (df['hdgree'] != 'not available') & (df['hdgree'] != 'not applicable')]

# Also filter for wages > 1
# NOTE: From the 2016 Census PUMF Guide: "Values that would have been rounded to zero have been
# replaced by 1" 
# Wages variable is rounded to base 1000 so someone who made 499 bucks would be logged as earning 1 
# someone earning 600 would be logged as earning 1000
df = df[(df['wages'] != 99999999) & (df['wages'] != 88888888)]  # these represent different not available codes
df = df[df['wages'] > 1]

# Filter for those working full time for 49 to 52 weeks
df = df[df['wrkact'] == 'worked 49 to 52 weeks full time']

# Filtering to only include immigrants and non-immigrants
df = df[(df['immstat'] == 'immigrants') | (df['immstat'] == 'non-immigrants')]


# Keep only QC, BC, ON, AB:
df = df[(df['pr'] == 'quebec') | (df['pr'] == 'ontario') | (df['pr'] == 'british columbia') | (df['pr'] == 'alberta')]


# Removing 'not available' entries for and Frenh at work questions:
df = df[df['lwafr'] != 'not available']


# Removing 'not available' entries for Visible Minority question:
df = df[df['vismin'] != 'not available']


# Removing those who immigrated before 1955 or who do not have their immigration year available:
df = df[df['yrimm'] != 'not available']

# Reset the index
df = df.reset_index(drop=True)


In [5]:
# Finding the 1st and 99th percentile of wage earners

df['wages'].quantile([0.01,0.99])

0.01      4000.0
0.99    355115.0
Name: wages, dtype: float64

In [6]:
# Checking how much 4000 wage accounts for the sample 


(df['wages'] == 4000).value_counts()[True] / len(df)

0.002354680403572408

In [7]:
# Checking how much 355115 wage accounts for the sample 


(df['wages'] == 355115).value_counts()[True] / len(df)

0.002621391776145335

In [8]:
# Winsorizing the data! 

df = df[(df['wages'] < 355115) & (df['wages'] > 4000)]
df = df.reset_index(drop=True)
df

,ppsort,weight,wt1,wt2,wt3,wt4,wt5,wt6,wt7,wt8,...,subsidy,tenur,totinc,totinc_at,value,vismin,wages,wkswrk,wrkact,yrimm
0,453141,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,not applicable,owned by a member of the household,97000,73000,450000,not a visible minority,95000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,not applicable
1,732612,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,not applicable,owned by a member of the household,46000,41000,839779,not a visible minority,19000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,not applicable
2,52611,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"no, not a subsidized dwelling",rented or band housing,53000,43000,99999999,not a visible minority,52000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,not applicable
3,700087,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,not applicable,owned by a member of the household,72000,60000,600000,not a visible minority,70000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,not applicable
4,753984,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,not applicable,owned by a member of the household,120000,98000,550000,multiple visible minorities,120000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,1980 to 1984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128320,595704,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,not applicable,owned by a member of the household,38000,35000,140000,not a visible minority,37000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,not applicable
128321,15851,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,not applicable,owned by a member of the household,130000,100000,860000,not a visible minority,140000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,not applicable
128322,756172,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,not applicable,owned by a member of the household,50000,40000,180000,not a visible minority,50000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,not applicable
128323,319614,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,not applicable,owned by a member of the household,64000,49000,390000,filipino,64000,49 to 52 weeks in 2015,worked 49 to 52 weeks full time,2000


## Part Two: Recoding

### a) Adding a quantitative variable that is the midpoint of the Age Group categorical variable

In [9]:
# Recoding: Defining a new variable that is the midpoint of the age groups

age_midpoints = []

for x in range(len(df['agegrp'])):
    first_number = df['agegrp'][x][0:2]
    after_to_key = df['agegrp'][x].index('to') + 3
    last_number = df['agegrp'][x][after_to_key:after_to_key+2]

    if ' ' in first_number:
        first_number = first_number.replace(' ','')
    
    if ' ' in last_number:
        last_number = last_number.replace(' ','')
    
    first_number = float(first_number)
    last_number = float(last_number)

    midpoint = (first_number + last_number)/2
    
    age_midpoints += [midpoint]
    
df['age midpoints'] = age_midpoints


### b) Simplifying 'hdgree' (highest degree) categorical variable from 15 categories to 7:
- Below High School
- High School 
- Below Bachelor 
- Bachelor 
- Above Bachelor
- not applicable
- not available

In [10]:
df['hdgree'] = df['hdgree'].replace("bachelor's degree","Bachelor").replace("no certificate, diploma or degree","Below High School")
df['hdgree'] = df['hdgree'].replace("secondary (high) school diploma or equivalency certificate","High School")
df['hdgree'] = df['hdgree'].replace("trades certificate or diploma other than certificate of apprenticeship or certificate of qualification","Below Bachelor")
df['hdgree'] = df['hdgree'].replace("certificate of apprenticeship or certificate of qualification","Below Bachelor")
df['hdgree'] = df['hdgree'].replace("program of 1 to 2 years (college, cegep and other non-university certificates or diplomas)","Below Bachelor")
df['hdgree'] = df['hdgree'].replace("program of more than 2 years (college, cegep and other non-university certificates or diplomas)","Below Bachelor")
df['hdgree'] = df['hdgree'].replace("master's degree","Above Bachelor")
df['hdgree'] = df['hdgree'].replace("program of 3 months to less than 1 year (college, cegep and other non-university certificates or diplomas)","Below Bachelor")
df['hdgree'] = df['hdgree'].replace("university certificate or diploma below bachelor level","Below Bachelor")
df['hdgree'] = df['hdgree'].replace("university certificate or diploma above bachelor level","Above Bachelor")
df['hdgree'] = df['hdgree'].replace("earned doctorate","Above Bachelor").replace("degree in medicine, dentistry, veterinary medicine or optometry","Above Bachelor")
df['hdgree'].value_counts()


Below Bachelor       70013
Bachelor             40522
Above Bachelor       17790
Below High School        0
High School              0
not available            0
not applicable           0
Name: hdgree, dtype: int64

### c) Simplifying 'locstud' (Location of study) categorical variable so that all the values for provinces (e.g. 'ontario' or 'alberta') are recoded as simply 'Canada'

In [11]:
df['locstud'] = df['locstud'].replace("ontario", "Canada").replace("quebec","Canada").replace("british columbia","Canada").replace("alberta","Canada").replace("manitoba","Canada")
df['locstud'] = df['locstud'].replace("saskatchewan","Canada").replace("newfoundland and labrador","Canada").replace("new brunswick","Canada").replace("prince edward island","Canada")
df['locstud'] = df['locstud'].replace("northern canada","Canada").replace("nova scotia","Canada")
df['locstud'] = df['locstud'].cat.remove_unused_categories()
df['locstud'].value_counts()

Canada                         111671
southeast and southern asia      5146
europe                           4401
united states                    2349
other countries and regions      1908
eastern asia                     1798
other americas                   1052
Name: locstud, dtype: int64

### d) Changing year of immigration to contain two values: either "Established" (arrived in 2000 or earlier) or "Recent" (arrived in 2001 or later)

This categorization split on these years was based on my supervising prof's (Dr. Nicole Fortin, a verifiable expert on economics of immigration in Canada) recommendation.

In [12]:
df['yrimm'] = df['yrimm'].replace("1985 to 1989","Established").replace("1980 to 1984","Established").replace("1975 to 1979","Established").replace("1970 to 1974","Established").replace("1965 to 1969", "Established").replace("2015 to 2016","Recent").replace("1960 to 1964","Established").replace("1955 to 1959","Established")
df['yrimm'] = df['yrimm'].replace("2001","Recent").replace("2000","Established").replace("2002","Recent").replace("2005","Recent")
df['yrimm'] = df['yrimm'].replace("2004","Recent").replace("2003","Recent").replace("1996","Established").replace("2010","Recent")
df['yrimm'] = df['yrimm'].replace("2007","Recent").replace("1990","Established").replace("1999","Established").replace("2006","Recent")
df['yrimm'] = df['yrimm'].replace("2009","Recent").replace("1997","Established").replace("2008","Recent").replace("1994","Established")
df['yrimm'] = df['yrimm'].replace("1995","Established").replace("1993","Established").replace("1992","Established").replace("1998","Established").replace("2011","Recent")
df['yrimm'] = df['yrimm'].replace("2012","Recent").replace("1991","Established").replace("2013","Recent").replace("2014","Recent")
df['yrimm'] = df['yrimm'].replace("before 1955","Established")
df['yrimm'] = df['yrimm'].cat.remove_unused_categories()
print(df['yrimm'].value_counts())


not applicable    98157
Established       17347
Recent            12821
Name: yrimm, dtype: int64


In [13]:
# filtering for people who either said yes to french at work or said no to french at work 

df = df[(df['lwafr'].str.contains("false")) | (df['lwafr'].str.contains("true"))]
df['lwafr'] = df['lwafr'].cat.remove_unused_categories()

In [14]:
df['pr'] = df['pr'].replace("ontario", "anglo-can").replace("alberta","anglo-can").replace("british columbia","anglo-can")
df['pr'] = df['pr'].cat.remove_unused_categories()
df['pr'].value_counts()

anglo-can    92960
quebec       35365
Name: pr, dtype: int64

In [15]:
df['immstat'] = df['immstat'].cat.remove_unused_categories()

### df2 will be used for taking the logarithm of wages - I wanted to keep the other df for calculating some descriptive statistics and thought that keeping wages as they were given in the PUMF might be more useful for descriptive statistics

#### * so let's use df2 for the regressions then cause it has wages in logarithms *

In [16]:
# First just going to drop some unused categories within the different variables:
df['hdgree'] = df['hdgree'].cat.remove_unused_categories()
df['cma'] = df['cma'].cat.remove_unused_categories()
df['pob'] = df['pob'].cat.remove_unused_categories()
# while it would be interesting to examine how outcomes difffer across differnt visible minority groups 
# for the purposes of this research, I'm keeping things relatively simple and just having two categories
# not a visible minority and visible minority
df['vismin'] = df['vismin'].replace('south asian', 'minority').replace("chinese","minority").replace("black",'minority').replace("filipino","minority").replace("latin american","minority").replace("arab","minority").replace("southeast asian","minority").replace("west asian","minority").replace("multiple visible minorities","minority").replace("visible minority, n.i.e.","minority").replace("japanese","minority").replace("korean","minority")
df['vismin'] = df['vismin'].cat.remove_unused_categories()



In [17]:
df2 = df.copy()

In [18]:
df2['wages'] = np.log(df2['wages'])

## Part Three: Calculating some descriptive statistics for the filtered data

### a) Descriptive statistics for the filtered data:

In [19]:
# Defining the sub-samples of the data I want descriptive statistics for:

anglo_canada = df[df['pr'] == 'anglo-can']
quebec = df[df['pr'] == 'quebec']

anglo_can_nat = anglo_canada[anglo_canada['immstat'] == 'non-immigrants']
anglo_can_imm = anglo_canada[anglo_canada['immstat'] == 'immigrants']

qc_nat = quebec[quebec['immstat'] == 'non-immigrants']
qc_imm = quebec[quebec['immstat'] == 'immigrants']


In [20]:
def descrip_stats(anglo_can_nat, string):
    '''
    This function takes in one of my sub-sample dataframes (anglo_can_nat, anglo_can_imm, qc_nat, qc_imm)
    and spits out some descriptive stats for that sub sample
    '''

    print(f'{string} Statistics:')
    print('---------------------------------------------------------------------------------')


    # Wages

    print('')
    print('Wages:')
    wg1 = round(anglo_can_nat['wages'].mean(),0)
    wg1_std = round(anglo_can_nat['wages'].std(),0)
    print(f'{wg1} ({wg1_std})')


    # Age

    age1 = round(anglo_can_nat['age midpoints'].mean(),2)
    age1_std = round(anglo_can_nat['age midpoints'].std(),2)
    print('\nAge:')
    print(f'{age1} ({age1_std})')


    # Mother Tongue

    eng1 = round(anglo_can_nat['mtnen'].value_counts()['true - respondent reported english as mother tongue']/len(anglo_can_nat),2)
    frn1 = round(anglo_can_nat['mtnfr'].value_counts()['true - respondent reported french as mother tongue']/len(anglo_can_nat),2)
    othr1 = 1 - eng1 - frn1
    print('\nMother Tongue:')
    print(f'English: {eng1}     French:{frn1}     Other:{round(othr1,2)}') 


    # Visible minority

    vismin1 = round((anglo_can_nat['vismin'] == 'not a visible minority').value_counts()[False] / len(anglo_can_nat),2)
    print('\nVisible Minority:')
    print(f'{vismin1}')


    # Highest Degree

    print('\nHighest Degree:')
    blw_bac1 = round(anglo_can_nat['hdgree'].value_counts()['Below Bachelor']/ len(anglo_can_nat),3)
    bac1 = round(anglo_can_nat['hdgree'].value_counts()['Bachelor']/ len(anglo_can_nat),3)
    abv_bac1 = round(anglo_can_nat['hdgree'].value_counts()['Above Bachelor']/ len(anglo_can_nat),3)
    print(f'Below Bachelor: {blw_bac1}     Bachelor: {bac1}     Above Bachelor: {abv_bac1}')


    # Places of study

    cn_stud = round(anglo_can_nat['locstud'].value_counts()['Canada'] / len(anglo_can_nat),3)
    us_stud = round(anglo_can_nat['locstud'].value_counts()['united states'] / len(anglo_can_nat),3)
    eu_stud = round(anglo_can_nat['locstud'].value_counts()['europe'] / len(anglo_can_nat),3)
    other_stud = round(anglo_can_nat['locstud'].value_counts()['other countries and regions'] / len(anglo_can_nat),3)
    other_amer_stud = round(anglo_can_nat['locstud'].value_counts()['other americas'] / len(anglo_can_nat),3)
    se_asia_stud = round(anglo_can_nat['locstud'].value_counts()['southeast and southern asia'] / len(anglo_can_nat),3)
    east_asia_stud = round(anglo_can_nat['locstud'].value_counts()['eastern asia'] / len(anglo_can_nat),3)
    print('\nLocation of Study:')
    print(f'Canada: {cn_stud}     United States: {us_stud}     Europe: {eu_stud}     Other Americas: {other_amer_stud}')
    print(f'Eastern Asia: {east_asia_stud}     SE & South Asia: {se_asia_stud}     Other: {other_stud}')
    
    
    # No of Obs
    
    nobs = len(anglo_can_nat)
    print(f'\n\nNo. of Obs. {nobs}')
    
    print('---------------------------------------------------------------------------------')
    print('\n\n\n')

In [21]:
descrip_stats(anglo_can_nat, 'Anglophone Canada Native-Born')
descrip_stats(anglo_can_imm, 'Anglophone Canada Immigrant')
descrip_stats(qc_nat, 'Quebec Native-Born')
descrip_stats(qc_imm, 'Quebec Immigrant')

Anglophone Canada Native-Born Statistics:
---------------------------------------------------------------------------------

Wages:
76601.0 (45793.0)

Age:
41.71 (11.23)

Mother Tongue:
English: 0.9     French:0.05     Other:0.05

Visible Minority:
0.07

Highest Degree:
Below Bachelor: 0.574     Bachelor: 0.313     Above Bachelor: 0.113

Location of Study:
Canada: 0.972     United States: 0.019     Europe: 0.005     Other Americas: 0.0
Eastern Asia: 0.0     SE & South Asia: 0.0     Other: 0.003


No. of Obs. 67310
---------------------------------------------------------------------------------




Anglophone Canada Immigrant Statistics:
---------------------------------------------------------------------------------

Wages:
69717.0 (42047.0)

Age:
44.37 (10.35)

Mother Tongue:
English: 0.28     French:0.01     Other:0.71

Visible Minority:
0.7

Highest Degree:
Below Bachelor: 0.375     Bachelor: 0.398     Above Bachelor: 0.226

Location of Study:
Canada: 0.51     United States: 0.035

### b) Countries and regions of origins of recent (arrived in 2001 or later) immigrants by province

And more!

In [22]:
# Just checking to see where the recent immigrants are from in 2016:

# First I'm filtering for immigrants that arrived from 2001 to 2016:

n = 10 # Show the top n countries

# Note that this is with the 'census' dataframe (i.e. the unfiltered dataframe)

recent_immis = census[(census['yrimm'] == '2015 to 2016') | (census['yrimm'] == '2014') | (census['yrimm'] == '2013')
      | (census['yrimm'] == '2012') | (census['yrimm'] == '2011') | (census['yrimm'] == '2010')
       | (census['yrimm'] == '2009') | (census['yrimm'] == '2008') | (census['yrimm'] == '2007')
       | (census['yrimm'] == '2006')| (census['yrimm'] == '2005') | (census['yrimm'] == '2004')
        | (census['yrimm'] == '2003') | (census['yrimm'] == '2002') | (census['yrimm'] == '2001')]

can_recent_immis = recent_immis[(recent_immis['pr'] != 'quebec') & (recent_immis['pr'] != 'new brunswick') & (recent_immis['pr'] != 'northern canada')]
can_recent_immis = can_recent_immis['pob'].value_counts()
print(f'Top {n} sources of immigrants for Anglophone Canada (1997-2016):\n---------------------------------------------------\n{can_recent_immis[0:n]}\n---------------------------------------------------\n\n')


on_recent_immis = recent_immis[recent_immis['pr'] == 'ontario']
qc_recent_immis = recent_immis[recent_immis['pr'] == 'quebec']
bc_recent_immis = recent_immis[recent_immis['pr'] == 'british columbia']
ab_recent_immis = recent_immis[recent_immis['pr'] == 'alberta']

on_recent_immis = on_recent_immis['pob'].value_counts()
print(f'Top {n} sources of immigrants for Ontario (1997-2016):\n---------------------------------------------------\n{on_recent_immis[0:n]}\n---------------------------------------------------\n\n')


qc_recent_immis = qc_recent_immis['pob'].value_counts()
print(f'Top {n} sources of immigrants for Quebec (1997-2016):\n---------------------------------------------------\n{qc_recent_immis[0:n]}\n---------------------------------------------------\n\n')


bc_recent_immis = bc_recent_immis['pob'].value_counts()
print(f'Top {n} sources of immigrants for British Columbia (1997-2016):\n---------------------------------------------------\n{bc_recent_immis[0:n]}\n---------------------------------------------------\n\n')


ab_recent_immis = ab_recent_immis['pob'].value_counts()
print(f'Top {n} sources of immigrants for Alberta (1997-2016):\n---------------------------------------------------\n{ab_recent_immis[0:n]}\n---------------------------------------------------\n\n')


Top 10 sources of immigrants for Anglophone Canada (1997-2016):
---------------------------------------------------
india                                          9780
philippines                                    9672
china                                          9250
other west central asia and the middle east    5387
other eastern europe                           3317
pakistan                                       3075
not available                                  2838
south america                                  2448
other africa                                   1995
iran                                           1987
Name: pob, dtype: int64
---------------------------------------------------


Top 10 sources of immigrants for Ontario (1997-2016):
---------------------------------------------------
india                                          5501
china                                          4917
other west central asia and the middle east    3835
philippines             

In [23]:
# Converting everything from the last cell into a percentage of total immigrants during that time period for each region
# In retrospect I could've just made a function for this... 
# Alberta:
ab_recent = ab_recent_immis.reset_index()
ab_sum = sum(ab_recent['pob'])
percent_ab = []
for x in ab_recent['pob']:
    j = x/ab_sum
    j = round(j,3)*100
    percent_ab += [j]
ab_recent['percent'] = percent_ab

# Ontario
on_recent = on_recent_immis.reset_index()
on_sum = sum(on_recent['pob'])
percent_on = []
for x in on_recent['pob']:
    j = x/on_sum
    j = round(j,3)*100
    percent_on += [j]
on_recent['percent'] = percent_on

# British Columbia
bc_recent = bc_recent_immis.reset_index()
bc_sum = sum(bc_recent['pob'])
percent_bc = []
for x in bc_recent['pob']:
    j = x/bc_sum
    j = round(j,3)*100
    percent_bc += [j]
bc_recent['percent'] = percent_bc

# Quebec
qc_recent = qc_recent_immis.reset_index()
qc_sum = sum(qc_recent['pob'])
percent_qc = []
for x in qc_recent['pob']:
    j = x/qc_sum
    j = round(j,3)*100
    percent_qc += [j]
qc_recent['percent'] = percent_qc

# Anglophone Canada
can_recent = can_recent_immis.reset_index()
can_sum = sum(can_recent['pob'])
percent_can = []
for x in can_recent['pob']:
    j = x/can_sum
    j = round(j,3)*100
    percent_can += [j]
can_recent['percent'] = percent_can

print('Alberta Recent Immigrants:')
print(ab_recent)
print('--------------------------------------------------------------')
print('BC Recent Immigrants')
print(bc_recent)
print('--------------------------------------------------------------')
print('Ontario Recent Immigrants')
print(on_recent)
print('--------------------------------------------------------------')
print('Quebec Recent Immigrants')
print(qc_recent)
print('--------------------------------------------------------------')
print('Canada Recent Immigrants')
print(can_recent)

Alberta Recent Immigrants:
                                          index   pob  percent
0                                   philippines  2636     21.6
1                                         india  1683     13.8
2                                         china   907      7.4
3   other west central asia and the middle east   739      6.1
4                                eastern africa   718      5.9
5                                  other africa   603      4.9
6                                 not available   561      4.6
7                                      pakistan   524      4.3
8                                 south america   499      4.1
9                          other eastern europe   471      3.9
10                               united kingdom   416      3.4
11                              central america   388      3.2
12                                united states   311      2.5
13                              northern africa   226      1.9
14                          

In [24]:
# Here I would like to calculate the unemployment rate for immigrants versus non-immigrants

# First on a national basis

labour_force = census[census['lfact'].str.contains('employed')]
labour_force = labour_force[labour_force['immstat'] != 'not available']

immigrant_force = labour_force[labour_force['immstat'] == 'immigrants']
native_force = labour_force[labour_force['immstat'] == 'non-immigrants']


cdn_imm_unemployment = (round((len(immigrant_force[immigrant_force['lfact'].str.contains('unemployed')])) / len(immigrant_force),3))*100
print(f'The unemployment rate for IMMIGRANTS NATIONALLY in 2016 was {cdn_imm_unemployment}%')

cdn_nativ_unemployment = (round((len(native_force[native_force['lfact'].str.contains('unemployed')])) / len(native_force),3))*100
print(f'The unemployment rate for NATIVE-BORN Canadians NATIONALLY in 2016 was {cdn_nativ_unemployment}%\n\n')

# Next let's do Ontario

on_imm_force = immigrant_force[immigrant_force['pr'] == 'ontario']
on_imm_unemployment = (round((len(on_imm_force[on_imm_force['lfact'].str.contains('unemployed')])) / len(on_imm_force),3))*100
print(f'The unemployment rate for IMMIGRANTS in ONTARIO in 2016 was {on_imm_unemployment}%')

on_nat_force = native_force[native_force['pr'] == 'ontario']
on_nat_unemployment = (round((len(on_nat_force[on_nat_force['lfact'].str.contains('unemployed')])) / len(on_nat_force),4))*100
print(f'The unemployment rate for NATIVE-BORN Canadians in ONTARIO in 2016 was {on_nat_unemployment}%\n\n')


# Okay now on to Quebec

qc_imm_force = immigrant_force[immigrant_force['pr'] == 'quebec']
qc_imm_unemployment = (round((len(qc_imm_force[qc_imm_force['lfact'].str.contains('unemployed')])) / len(qc_imm_force),3))*100
print(f'The unemployment rate for IMMIGRANTS in QUEBEC in 2016 was {qc_imm_unemployment}%')

qc_nat_force = native_force[native_force['pr'] == 'quebec']
qc_nat_unemployment = (round((len(qc_nat_force[qc_nat_force['lfact'].str.contains('unemployed')])) / len(qc_nat_force),4))*100
print(f'The unemployment rate for NATIVE-BORN Canadians in QUEBEC in 2016 was {qc_nat_unemployment}%\n\n')





The unemployment rate for IMMIGRANTS NATIONALLY in 2016 was 7.6%
The unemployment rate for NATIVE-BORN Canadians NATIONALLY in 2016 was 7.7%


The unemployment rate for IMMIGRANTS in ONTARIO in 2016 was 7.3%
The unemployment rate for NATIVE-BORN Canadians in ONTARIO in 2016 was 7.39%


The unemployment rate for IMMIGRANTS in QUEBEC in 2016 was 9.6%
The unemployment rate for NATIVE-BORN Canadians in QUEBEC in 2016 was 6.67%




In [25]:
# Just want to know how much each province accounts for the total of immigration to Canada:

census2 = census.copy()
census2 = census2[(census2['immstat'] == 'immigrants')]
print(len(census2[census2['pr'] == 'british columbia']) / len(census2))
print(len(census2[census2['pr'] == 'alberta']) / len(census2))
print(len(census2[census2['pr'] == 'ontario']) / len(census2))
print(len(census2[census2['pr'] == 'quebec']) / len(census2))


0.1719602609727165
0.11216884143930407
0.512144128113879
0.14454329774614472


In [26]:
# And how much each province accounts for the total immigration to Anglo Canada:

census3 = census2.copy()
census3 = census3[census3['pr'] != 'northern canada']
census3 = census3[census3['pr'] != 'quebec']
census3 = census3[census3['pr'] != 'new brunswick']
census3 = census3[(census3['immstat'] == 'immigrants')]

print(len(census3[census3['pr'] == 'british columbia']) / len(census3))
print(len(census3[census3['pr'] == 'alberta']) / len(census3))
print(len(census3[census3['pr'] == 'ontario']) / len(census3))


0.20219684421584866
0.1318920175515067
0.6021968442158486


In [27]:
# Checking to see what % of New Brunswick speaks French as their first language
nwbrnswick_lang = census[census['pr'] == 'new brunswick']
nwbrnswick_lang = nwbrnswick_lang[nwbrnswick_lang['mtnfr'] != 'not available']

print('This is the percentage (in decimal format) of people in New Brunswick\nwho speak French as their Mother Tongue:')
nwbrnswick_lang['mtnfr'].value_counts()['true - respondent reported french as mother tongue'] / len(nwbrnswick_lang)

This is the percentage (in decimal format) of people in New Brunswick
who speak French as their Mother Tongue:


0.3234276409525748

In [28]:
# Checking to see what % of Northern Canada speaks Other as their first language
north_lang = census[census['pr'] == 'northern canada']
north_lang = north_lang[north_lang['mtnfr'] != 'not available']
north_lang = north_lang[north_lang['mtnen'] != 'not available']

print('This is the percentage (in decimal format) of people in Northern Canada\nwho speak a language besides English or French as their Mother Tongue:')
north_eng = north_lang['mtnen'].value_counts()['true - respondent reported english as mother tongue']
north_fr = north_lang['mtnfr'].value_counts()['true - respondent reported french as mother tongue']
1 - ((north_eng + north_fr) / len(north_lang))

This is the percentage (in decimal format) of people in Northern Canada
who speak a language besides English or French as their Mother Tongue:


0.30932346011443956

## PART FOUR: Finally doing some regressions!
### and doing some last minute prep for my dataframes that I'm going to be running regressions with

In [29]:
df3 = df2.copy()
df3 = df3[df3['immstat'] == 'immigrants']
df3 = df3.reset_index(drop=True)
df3['agesb'] = list(df3['age midpoints'])

In [30]:
df2['lwafr'] = df2['lwafr'].replace("false - respondent did not report french as the language used most often at work","false").replace("true - respondent reported french as the language used most often at work","true")

In [31]:
df2['hlbfr'] = df2['hlbfr'].cat.remove_unused_categories()

In [32]:
df2['hlbfr'] = df2['hlbfr'].replace("false - respondent did not report french as the language spoken at home on a regular basis","false").replace("true - respondent reported french as the language spoken at home on a regular basis","true")
df2['hlbfr'].value_counts()

false    124839
true       3486
Name: hlbfr, dtype: int64

In [33]:
df2['hlafr'] = df2['hlafr'].cat.remove_unused_categories()
df2['hlafr'] = df2['hlafr'].replace("false - respondent did not report french as the language spoken most often at home","false").replace("true - respondent reported french as the language spoken most often at home","true")
df2['hlafr'].value_counts()

false    96682
true     31643
Name: hlafr, dtype: int64

In [34]:
df2['mtnfr'] = df2['mtnfr'].cat.remove_unused_categories()
df2['mtnfr'] = df2['mtnfr'].replace("false - respondent did not report french as mother tongue","false").replace("true - respondent reported french as mother tongue","true")
df2['mtnfr'].value_counts()

false    96186
true     32139
Name: mtnfr, dtype: int64

In [35]:
df5 = df2.copy()

In [36]:
df5['hlafr'] = df5.hlafr.astype(str)
df5['hlbfr'] = df5.hlbfr.astype(str)
df5['mtnfr'] = df5.mtnfr.astype(str)
df5['french'] = df5['hlafr'] + df5['hlbfr'] + df5['mtnfr']

        
df5['french'] = df5['french'].replace('falsefalsefalse','false')
df5['french'] = (df5['french']  != 'false')
df5['french'] = df5.french.astype(str)
df5['french']

0         False
1         False
2          True
3         False
4         False
          ...  
128320    False
128321    False
128322     True
128323    False
128324    False
Name: french, Length: 128325, dtype: object

In [38]:
fit = ols('wages ~ C(cma, Treatment("toronto")) + age_midpoints + np.power(age_midpoints, 2) + C(sex, Treatment("male")) +C(yrimm, Treatment("not applicable")) + C(locstud, Treatment("Canada"))*C(pr, Treatment("anglo-can")) + C(hdgree, Treatment("Below Bachelor"))  + C(vismin, Treatment("not a visible minority"))' , data=df2).fit()

fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  wages   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     955.7
Date:                Wed, 01 Dec 2021   Prob (F-statistic):               0.00
Time:                        16:18:16   Log-Likelihood:            -1.0209e+05
No. Observations:              128325   AIC:                         2.043e+05
Df Residuals:                  128284   BIC:                         2.047e+05
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
================================================================================================================================================================================
                                                                                                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                        8.9825      0.022    400.759      0.000       8.939       9.026
C(cma, Treatment("toronto"))[T.québec]                                                                          -0.0034      0.011     -0.310      0.757      -0.025       0.018
C(cma, Treatment("toronto"))[T.montréal]                                                                         0.0384      0.008      4.667      0.000       0.022       0.055
C(cma, Treatment("toronto"))[T.sherbrooke  trois-rivières]                                                     -0.0603      0.016     -3.877      0.000      -0.091      -0.030
C(cma, Treatment("toronto"))[T.ottawa  gatineau]                                                                0.0370      0.008      4.908      0.000       0.022       0.052
C(cma, Treatment("toronto"))[T.oshawa]                                                                           0.0020      0.014      0.143      0.886      -0.025       0.029
C(cma, Treatment("toronto"))[T.hamilton]                                                                        -0.0347      0.010     -3.436      0.001      -0.055      -0.015
C(cma, Treatment("toronto"))[T.st. catharines  niagara]                                                        -0.1346      0.015     -9.130      0.000      -0.163      -0.106
C(cma, Treatment("toronto"))[T.kitchener  cambridge  waterloo]                                                -0.0376      0.012     -3.164      0.002      -0.061      -0.014
C(cma, Treatment("toronto"))[T.london]                                                                          -0.1214      0.013     -9.633      0.000      -0.146      -0.097
C(cma, Treatment("toronto"))[T.windsor]                                                                         -0.0766      0.016     -4.804      0.000      -0.108      -0.045
C(cma, Treatment("toronto"))[T.brantford  guelph  barrie]                                                     -0.0667      0.013     -5.297      0.000      -0.091      -0.042
C(cma, Treatment("toronto"))[T.kingston  peterborough]                                                         -0.1233      0.017     -7.362      0.000      -0.156      -0.090
C(cma, Treatment("toronto"))[T.greater sudbury / grand sudbury  thunder bay]                                   -0.0492      0.016     -3.102      0.002      -0.080      -0.018
C(cma, Treatment("toronto"))[T.calgary]                                   

In [39]:
fit = ols('wages ~ C(cma, Treatment("toronto")) + age_midpoints + np.power(age_midpoints, 2) + C(sex, Treatment("male")) +C(yrimm, Treatment("not applicable")) + C(locstud, Treatment("Canada"))*C(pr, Treatment("anglo-can"))*C(french, Treatment("False")) + C(hdgree, Treatment("Below Bachelor"))  + C(vismin, Treatment("not a visible minority"))' , data=df5).fit()

fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  wages   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     709.1
Date:                Wed, 01 Dec 2021   Prob (F-statistic):               0.00
Time:                        16:18:39   Log-Likelihood:            -1.0207e+05
No. Observations:              128325   AIC:                         2.042e+05
Df Residuals:                  128270   BIC:                         2.048e+05
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
================================================================================================================================================================================================================
                                                                                                                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                                        8.9826      0.022    400.796      0.000       8.939       9.027
C(cma, Treatment("toronto"))[T.québec]                                                                                                          -0.0043      0.011     -0.391      0.696      -0.026       0.017
C(cma, Treatment("toronto"))[T.montréal]                                                                                                         0.0424      0.008      5.087      0.000       0.026       0.059
C(cma, Treatment("toronto"))[T.sherbrooke  trois-rivières]                                                                                     -0.0609      0.016     -3.911      0.000      -0.091      -0.030
C(cma, Treatment("toronto"))[T.ottawa  gatineau]                                                                                                0.0375      0.008      4.876      0.000       0.022       0.053
C(cma, Treatment("toronto"))[T.oshawa]                                                                                                           0.0024      0.014      0.171      0.865      -0.025       0.030
C(cma, Treatment("toronto"))[T.hamilton]                                                                                                        -0.0346      0.010     -3.419      0.001      -0.054      -0.015
C(cma, Treatment("toronto"))[T.st. catharines  niagara]                                                                                        -0.1343      0.015     -9.108      0.000      -0.163      -0.105
C(cma, Treatment("toronto"))[T.kitchener  cambridge  waterloo]                                                                                -0.0372      0.012     -3.131      0.002      -0.060      -0.014
C(cma, Treatment("toronto"))[T.london]                                                                                                          -0.1211      0.013     -9.609      0.000      -0.146      -0.096
C(cma, Treatment("toronto"))[T.windsor]                                                                                                         -0.0763      0.016     -4.785      0.000      -0.108      -0.045
C(cma, Treatment("toronto"))[T.brantford  guelph  barrie]                                                                                     -0.0663      

In [40]:
fit = ols('wages ~ age_midpoints + np.power(age_midpoints, 2) + C(locstud, Treatment("Canada"))*C(pr, Treatment("anglo-can")) + C(hdgree, Treatment("Below Bachelor"))  + C(yrimm, Treatment("not applicable"))' , data=df2).fit()

fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  wages   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                     1431.
Date:                Wed, 01 Dec 2021   Prob (F-statistic):               0.00
Time:                        16:19:08   Log-Likelihood:            -1.0650e+05
No. Observations:              128325   AIC:                         2.130e+05
Df Residuals:                  128305   BIC:                         2.132e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================================================================
                                                                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                  8.8637      0.023    387.083      0.000       8.819       8.909
C(locstud, Treatment("Canada"))[T.united states]                                                          -0.0483      0.012     -3.929      0.000      -0.072      -0.024
C(locstud, Treatment("Canada"))[T.other americas]                                                         -0.1825      0.021     -8.725      0.000      -0.223      -0.141
C(locstud, Treatment("Canada"))[T.europe]                                                                 -0.0596      0.010     -5.679      0.000      -0.080      -0.039
C(locstud, Treatment("Canada"))[T.eastern asia]                                                           -0.3179      0.014    -22.023      0.000      -0.346      -0.290
C(locstud, Treatment("Canada"))[T.southeast and southern asia]                                            -0.3365      0.010    -35.332      0.000      -0.355      -0.318
C(locstud, Treatment("Canada"))[T.other countries and regions]                                            -0.1469      0.015     -9.501      0.000      -0.177      -0.117
C(pr, Treatment("anglo-can"))[T.quebec]                                                                   -0.1929      0.004    -52.574      0.000      -0.200      -0.186
C(hdgree, Treatment("Below Bachelor"))[T.Bachelor]                                                         0.2514      0.004     70.904      0.000       0.244       0.258
C(hdgree, Treatment("Below Bachelor"))[T.Above Bachelor]                                                   0.3900      0.005     80.295      0.000       0.381       0.400
C(yrimm, Treatment("not applicable"))[T.Established]                                                      -0.0707      0.005    -14.067      0.000      -0.081      -0.061
C(yrimm, Treatment("not applicable"))[T.Recent]                                                           -0.1674      0.007    -23.795      0.000      -0.181      -0.154
C(locstud, Treatment("Canada"))[T.united states]:C(pr, Treatment("anglo-can"))[T.quebec]                   0.1058      0.045      2.361      0.018       0.018       0.194
C(locstud, Treatment("Canada"))[T.other americas]:C(pr, Treatment("anglo-can"))[T.quebec]                 -0.0517      0.038     -1.357      0.175      -0.126       0.023
C(locstud, Treatment("Canada"))[T.europe]:C(pr, Treatment("anglo-can"))[T.quebec]                          0.0027      0.020      0.134      0.893      -0.037       0.043
C(loc

In [41]:
fit = ols('wages ~ C(cma, Treatment("toronto")) + age_midpoints + np.power(age_midpoints, 2) + C(sex, Treatment("male")) +C(yrimm, Treatment("not applicable")) + C(locstud, Treatment("Canada"))*C(pr, Treatment("anglo-can"))*C(vismin, Treatment("not a visible minority")) + C(hdgree, Treatment("Below Bachelor")) ' , data=df5).fit()

fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  wages   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     722.8
Date:                Wed, 01 Dec 2021   Prob (F-statistic):               0.00
Time:                        16:22:42   Log-Likelihood:            -1.0206e+05
No. Observations:              128325   AIC:                         2.042e+05
Df Residuals:                  128271   BIC:                         2.048e+05
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================================================================================================================================
                                                                                                                                                                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                                                             8.9774      0.022    400.052      0.000       8.933       9.021
C(cma, Treatment("toronto"))[T.québec]                                                                                                                               -0.0009      0.011     -0.084      0.933      -0.022       0.021
C(cma, Treatment("toronto"))[T.montréal]                                                                                                                              0.0438      0.008      5.179      0.000       0.027       0.060
C(cma, Treatment("toronto"))[T.sherbrooke  trois-rivières]                                                                                                          -0.0583      0.016     -3.745      0.000      -0.089      -0.028
C(cma, Treatment("toronto"))[T.ottawa  gatineau]                                                                                                                     0.0392      0.008      5.182      0.000       0.024       0.054
C(cma, Treatment("toronto"))[T.oshawa]                                                                                                                                0.0042      0.014      0.302      0.763      -0.023       0.031
C(cma, Treatment("toronto"))[T.hamilton]                                                                                                                             -0.0328      0.010     -3.242      0.001      -0.053      -0.013
C(cma, Treatment("toronto"))[T.st. catharines  niagara]                                                                                                             -0.1321      0.015     -8.950      0.000      -0.161      -0.103
C(cma, Treatment("toronto"))[T.kitchener  cambridge  waterloo]                                                                                                     -0.0350      0.012     -2.939      0.003      -0.058      -0.012
C(cma, Treatment("toronto"))[T.london]                                                                                                                               -0.1190      0.013     -9.430      0.000      -0.144      -0.094
C(cma, Treatment("toronto"))[T.windsor]                                                      

## Just a final note here:

### All this code makes more sense when considered in the context of my actual research paper. My intention here was not to explain every line of code and every regression, but rather just to show the relevant Python work associated with my research paper and to demonstrate that I have an incredibly thorough understanding of pandas, numpy, and more.